In [82]:
#librerias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [83]:
#cargar datos
#importamos a un dataframe

df = pd.read_csv(r"C:\Users\ninic\OneDrive - Lambton College\Term 1\BDM 1113-NoSQL data Base\project\Project-Data-Analysis-Lambton\Electricity_Generation.csv", header=None, names=range(47))
#nombre de las tablas
table_names = ["Canada", "Newfoundland and Labrador", "Prince Edward Island", "Nova Scotia", "New Brunswick","Quebec", "Ontario","Manitoba", "Alberta", "British Columbia", "Saskatchewan", "Yukon", "Northwest Territories", "Nunavut"]
# Identificar grupos basados en los nombres de las tablas
groups = df[0].isin(table_names).cumsum()
# Crear un diccionario de DataFrames para cada tabla
tables = {g.iloc[0,0]: g.iloc[1:-1, :20] for k,g in df.groupby(groups)} 


#acomodar datos
#selecciono una sola region 
df2 = tables["Canada"]
#eliminar primera fila porque los anos estaban en esa fila
df2 = df2.iloc[1:, :]
#eliminar esa columna
df2.drop(df2.columns[0], axis=1, inplace=True)
# Renombrar las columnas del 2005 al 2023
nuevos_nombres = {str(anio): f'Titulo_{anio}' for anio in range(2005, 2024)}
# Asignar nuevos nombres a las columnas
df2.columns = nuevos_nombres
#reset index
df2 = df2.reset_index(drop=True)
#nuevos nombres index
ind_nombres = ["Hydro / Wave / Tidal", "Wind", "Biomass / Geothermal","Solar","Uranium","Coal & Coke","Natural Gas","Oil"]  
df2.rename(index=dict(enumerate(ind_nombres)), inplace=True)
#transponer matriz
dataset = df2.T
#cambiar los numeros a fecha
dataset.index = pd.to_datetime(dataset.index, format='%Y')
#seleccionamos que solo sea WIND y lo pasamos de serie a dataframe 
dataset=dataset["Wind"].to_frame()
print(type(dataset))
#print(dataset)


<class 'pandas.core.frame.DataFrame'>


C:\Users\ninic\AppData\Local\Temp\ipykernel_45324\3663291827.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(df2.columns[0], axis=1, inplace=True)


In [86]:
df = dataset
df['Year'] = pd.to_datetime(df.index)
df.set_index('Year', inplace=True)

# Crear características basadas en valores anteriores
for i in range(1, 4):  # Usaremos los 3 años anteriores como características
    df[f'Wind_{i}'] = df['Wind'].shift(i)

"""
Usar los 3 años anteriores como características significa que, para predecir el valor de la variable objetivo (en este caso, la generación de energía eólica) de un año específico,
 utilizamos los valores de esa misma variable de los tres años anteriores como datos de entrada para el modelo."""

df.dropna(inplace=True)  # Eliminar filas con valores NaN resultantes de los shifts

# Dividir los datos en conjuntos de entrenamiento y prueba
X = df.drop(['Wind'], axis=1)
y = df['Wind']

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Random Forest
model = RandomForestRegressor(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

print(type(y_pred))

<class 'numpy.ndarray'>
